In [15]:
import shutil
import os
import xml.etree.ElementTree as ET
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

kaggle dataset link: https://www.kaggle.com/datasets/team-ai/japaneseenglish-bilingual-corpus?resource=download  
This particular dataset is translated by a human and even checked.

In [2]:
# code to extract the zip file
data_folder_name=r'D:\chronicles\eng-jap-data'
target_dir=r'archive.zip'
shutil.unpack_archive(target_dir,data_folder_name)

This praticular dataset is stored in the form of XML.  
Where <j> tag has the japanese sentence and <e> has the english sentence.  
I have only considerd the english sentences which is checked rather than sentences which is just translated to english.

In [ ]:
os.remove('D:\chronicles\eng-jap-data\wiki_corpus_2.01\GNM\GNM00155.xml')

In the below cell i am extracting the value of <j> and <e> tags and storing it in CSV file using pandas.

In [20]:
col_names=['japanese_sentence','english_sentence']
df=pd.DataFrame()
directory='D:\chronicles\eng-jap-data\wiki_corpus_2.01'
for item in os.listdir(directory):
    item=os.path.join(directory,item)
    print("In {} folder".format(item))
    if os.path.isdir(item):
        for xml_file in os.listdir(item):
            xml_file_path=os.path.join(item,xml_file)
            tree=ET.parse(xml_file_path)
            root=tree.getroot()
            for sen in root.findall("par/sen/[@id]"):
                jap_sentence=sen.find("./j").text
                eng_sentence=sen.find("./e/[@type='check']").text
                row=pd.Series([jap_sentence,eng_sentence])
                df=df.append(row,ignore_index=True)
print("Extracting done!")

In D:\chronicles\eng-jap-data\wiki_corpus_2.01\BDS folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\BLD folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\CLT folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\EPR folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\FML folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\GNM folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\HST folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\kyoto_lexicon.csv folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\LTT folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\PNM folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\readme.pdf folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\RLW folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\ROD folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\SAT folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\SCL folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\SNT folder
In D:\chronicles\eng-jap-data\wiki_corpus_2.01\TTL 

In [22]:
df.columns=col_names
file_name='japtoeng.csv'
df.to_csv(file_name, sep='\t')
df

,japanese_sentence,english_sentence
0,雪舟（せっしゅう、1420年（応永27年） - 1506年（永正3年））は号で、15世紀後半...,"Known as Sesshu (1420 - 1506), he was an ink p..."
1,日本の水墨画を一変させた。,He revolutionized the Japanese ink painting.
2,諱は「等楊（とうよう）」、もしくは「拙宗（せっしゅう）」と号した。,"He was given the posthumous name ""Toyo"" or ""Se..."
3,備中国に生まれ、京都・相国寺に入ってから周防国に移る。,"Born in Bicchu Province, he moved to Suo Provi..."
4,その後遣明使に随行して中国（明）に渡って中国の水墨画を学んだ。,Later he accompanied a mission to Ming Dynasty...
...,...,...
167824,尚侍(ないしのかみ)とは、下記のとおりである。,Naishi no kami indicates as follows.
167825,1 明治・大正時代の、宮中女官の最高の官名。,1. the highest government post for the court l...
167826,2 平安時代の、内侍司の長官を勤めた女官の官名。,2. the government post for the court lady who ...
167827,天皇に近侍して、奏請・伝宣などをつかさどる。,"Naishi no kami attended on the emperor, being ..."
